# Tutorial: Batch Flight Requests
The `google_flights_scraper` package can also be used to make batch request of multiple flights where a certain criteria is varied (e.g. destination or dates)

## Set Up

In [1]:
# Import
from datetime import datetime, timedelta

from google_flights_scraper.jupyter_helper import (
    scrape_date_range,
    scrape_multiple_destinations,
)

# Get today's date
today = datetime.today()

## Destination Search
Search multiple destinations on certain dates. Helpful if you know when you want to travel, but don't know where and are looking for good deals. 


Below are the possible parameters when making a request:  

**Required Inputs**

`departure_code` : Departure airport or city (a single location)  
`departure_country` Departure country (a single country)  
`arrival_codes` : A list of possible destination airports or cities  
`arrival_countries` : The matching list of possible destination countries  
`start_date` : Departure date (in `"MM/DD/YYY"` format)
`end_date` : Arrrival date  (in `"MM/DD/YYY"` format)
`seat_classes` : The matching list of seat classes (reason this is `list` and not `str` is because domestic and international flights can have different class names)  

**Optional Inputs**  

`output_path` : path to output `.csv`  (ex: `export.csv`)
`delay_seconds`: How many seconds to delay between requests. Can be helpful to increase if you are being rate limited by Google.  

### Example: Departing from New York

In this example, we are departing from New York and searching for flights to SEA, London, Paris, and SFO.

In [2]:
# Search Mapping
city_search = {
    "SEA": {"country": "United States of America", "seat_class": "Economy (exclude Basic)"},
    "London": {"country": "United Kingdom", "seat_class": "Economy"},
    "Paris": {"country": "France", "seat_class": "Economy"},
    "SFO": {"country": "United States of America", "seat_class": "Economy (exclude Basic)"},
}

# Individual Lists
destination_codes = list(city_search.keys())
destination_countries = [city_search[code]["country"] for code in destination_codes]
seat_classes = [city_search[code]["seat_class"] for code in destination_codes]

# Create Dates
start = (today + timedelta(weeks=8)).strftime("%m/%d/%Y")
end = (today + timedelta(weeks=10)).strftime("%m/%d/%Y")

# Run
df_destinations = scrape_multiple_destinations(
    departure_code="New York",
    departure_country="United States of America",
    arrival_codes=destination_codes,
    arrival_countries=destination_countries,
    start_date=start,
    end_date=end,
    seat_classes=seat_classes,
    delay_seconds=3.0,
)

In [3]:
df_destinations

,departure_airport,departure_country,arrival_airport,arrival_country,departure_date,return_date,seat_class,departure_airline,departure_departure_airport,departure_departure_date,...,return_duration_minutes,return_duration_str,return_carry_on_bags,return_checked_bags,price,price_classification,price_difference,price_relativity,status,url
0,New York,United States of America,SEA,United States of America,04/05/2026,04/19/2026,Economy (exclude Basic),Alaska,John F. Kennedy International Airport,"Sunday, April 5",...,321,5 hr 21 min,1.0,0.0,396,low,406,0.5062,Ran successfully.,https://www.google.com/travel/flights/booking?...
1,New York,United States of America,SFO,United States of America,04/05/2026,04/19/2026,Economy (exclude Basic),JetBlue,John F. Kennedy International Airport,"Sunday, April 5",...,336,5 hr 36 min,1.0,0.0,467,low,476,0.5048,Ran successfully.,https://www.google.com/travel/flights/booking?...
2,New York,United States of America,Paris,France,04/05/2026,04/19/2026,Economy,American,John F. Kennedy International Airport,"Sunday, April 5",...,845,14 hr 5 min,NaN,NaN,471,low,315,0.4008,Ran successfully.,https://www.google.com/travel/flights/booking?...
3,New York,United States of America,London,United Kingdom,04/05/2026,04/19/2026,Economy,United,Newark Liberty International Airport,"Sunday, April 5",...,1155,19 hr 15 min,NaN,NaN,625,typical,0,0.0000,Ran successfully.,https://www.google.com/travel/flights/booking?...


This returns a pandas DataFrame of the best flight for each destination, sorted in order of `price_relativity` highlight the flights which are the "best deal".

This also contains all the other information returned by the basic flight request function.

## Date Range Search
Search all date combinations for a specific route between certain dates. Helpful if you're flexible on timing and looking for finding cheaper days to fly. 

Here are the possible parameters for this request:  

**Required Inputs**  

`departure_code` : Departure airport or city (a single location)  
`departure_country` : Departure country (a single country)  
`arrival_code` : Arrival airport or city (a single location)  
`arrival_country` : Arrival country (a single country)  
`start_date_range` : Start date for search (in `"MM/DD/YYY"` format)  
`end_date_range` : End date for search (in `"MM/DD/YYY"` format)  
`min_trip_length` : Minimum trip length (in days)  
`max_trip_length` : Maximum trip length (in days)  
`seat_class` : Class of seat (a single selection)  

**Optional Inputs**  

`output_path` : path to output `.csv`  (ex: `export.csv`)
`delay_seconds`: How many seconds to delay between requests. Can be helpful to increase if you are being rate limited by Google.  

### Example: JFK to London
In this example, we are departing from JFK and arriving in London anytime within a two-week period, searching from trip from 5-7 days long.

In [4]:
# Create Dates
start = (today + timedelta(weeks=11)).strftime("%m/%d/%Y")
end = (today + timedelta(weeks=12)).strftime("%m/%d/%Y")

# Run
df_dates = scrape_date_range(
    departure_code="JFK",
    departure_country="United States of America",
    arrival_code="London",
    arrival_country="United Kingdom",
    start_date_range=start,
    end_date_range=end,
    min_trip_length=5,
    max_trip_length=7,
    seat_class="Economy",
    delay_seconds=3.0,
)

In [5]:
df_dates

,departure_airport,departure_country,arrival_airport,arrival_country,departure_date,return_date,seat_class,departure_airline,departure_departure_airport,departure_departure_date,...,return_duration_str,return_carry_on_bags,return_checked_bags,price,price_classification,price_difference,price_relativity,status,url,trip_length_days
0,JFK,United States of America,London,United Kingdom,04/27/2026,05/02/2026,Economy,British Airways,John F. Kennedy International Airport,"Monday, April 27",...,13 hr 50 min,None,None,513,low,162,0.2400,Ran successfully.,https://www.google.com/travel/flights/booking?...,5
1,JFK,United States of America,London,United Kingdom,04/26/2026,05/01/2026,Economy,American,John F. Kennedy International Airport,"Sunday, April 26",...,10 hr 29 min,None,None,526,low,149,0.2207,Ran successfully.,https://www.google.com/travel/flights/booking?...,5
2,JFK,United States of America,London,United Kingdom,04/28/2026,05/03/2026,Economy,British Airways,John F. Kennedy International Airport,"Tuesday, April 28",...,13 hr 50 min,None,None,513,low,115,0.1831,Ran successfully.,https://www.google.com/travel/flights/booking?...,5
3,JFK,United States of America,London,United Kingdom,04/26/2026,05/02/2026,Economy,American,John F. Kennedy International Airport,"Sunday, April 26",...,13 hr 50 min,None,None,513,low,111,0.1779,Ran successfully.,https://www.google.com/travel/flights/booking?...,6
4,JFK,United States of America,London,United Kingdom,04/26/2026,05/03/2026,Economy,American,John F. Kennedy International Airport,"Sunday, April 26",...,13 hr 50 min,None,None,513,low,111,0.1779,Ran successfully.,https://www.google.com/travel/flights/booking?...,7
5,JFK,United States of America,London,United Kingdom,04/27/2026,05/03/2026,Economy,British Airways,John F. Kennedy International Airport,"Monday, April 27",...,13 hr 50 min,None,None,513,low,111,0.1779,Ran successfully.,https://www.google.com/travel/flights/booking?...,6


This also returns a pandas DataFrame of the best flight for each destination, sorted in order of `price_relativity` highlight the flights which are the "best deal".

This also contains all the other information returned by the basic flight request function.